# Diction Watermarking on Sentiment Classification Model

This notebook demonstrates Diction (activation-based) watermarking on a tiny BERT model.
Using `prajjwal1/bert-tiny` (4.4M params) for fast training.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F

# --- Classes Utilitaires ---

class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()


class ProjectionNet(nn.Module):
    """
    Reseau leger qui projette les features vers l'espace du watermark.
    Pour transformers: [Batch, SeqLen, Hidden] -> [Batch, WatermarkLen]
    """
    def __init__(self, hidden_size, watermark_len):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, watermark_len),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: [Batch, SeqLen, Hidden] -> Mean pooling sur SeqLen
        if len(x.shape) == 3:
            x = x.mean(dim=1)  # [Batch, Hidden]
        return self.net(x)


# --- Classe Principale Diction pour Sentiment ---

class DictionSentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # Configuration par defaut
        self.config = {
            "layer_name": "bert.encoder.layer.1.output",  # Derniere couche encoder
             # "layer_name": "classifier",
            "watermark_len": 64,
            "trigger_size": 32,  # Nombre d'exemples dans le trigger set
            "lr": 5e-5,
            "lr_proj": 1e-3,
            "lambda_wat": 10.0,
            "epochs": 10
        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles (modules):")
        for name, module in self.model.named_modules():
            if name:
                print(f"  {name}")
    
    def _get_target_layer(self, model, layer_name):
        """Recupere le module PyTorch correspondant au nom."""
        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")
    
    def create_trigger_set(self, dataset, tokenizer):
        """
        Cree un trigger set a partir d'un sous-ensemble du dataset.
        Le trigger set est un ensemble fixe d'inputs utilise pour l'embedding/extraction.
        """
        # Selectionner des indices aleatoires
        indices = torch.randperm(len(dataset))[:self.config["trigger_size"]].tolist()
        
        trigger_inputs = {
            "input_ids": [],
            "attention_mask": []
        }
        
        for idx in indices:
            item = dataset[idx]
            trigger_inputs["input_ids"].append(item["input_ids"])
            trigger_inputs["attention_mask"].append(item["attention_mask"])
        
        trigger_inputs["input_ids"] = torch.stack(trigger_inputs["input_ids"]).to(self.device)
        trigger_inputs["attention_mask"] = torch.stack(trigger_inputs["attention_mask"]).to(self.device)#combines the list of tensors into one single tensor
        
        return trigger_inputs
    
    def embed(self, train_dataloader, eval_dataloader, trigger_dataset):
        """
        Incorpore la marque Diction pendant le finetuning.
        """
        print(f"--- Demarrage Embedding DICTION ({self.config['layer_name']}) ---")
        
        # 1. Preparation des modeles
        original_model = deepcopy(self.model)
        original_model.eval()
        for p in original_model.parameters():
            p.requires_grad = False
        
        watermarked_model = self.model
        watermarked_model.train()
        
        # 2. Generation des cles et trigger set
        trigger_inputs = self.create_trigger_set(trigger_dataset, self.tokenizer)
        
        target_wm = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)
        random_wm = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)
        
        print(f"Trigger set size: {self.config['trigger_size']}")
        print(f"Watermark length: {self.config['watermark_len']} bits")
        
        # 3. Initialisation ProjectionNet (dimension dynamique)
        dummy_layer = self._get_target_layer(watermarked_model, self.config["layer_name"])
        dummy_hook = FeatureHook(dummy_layer)
        with torch.no_grad():
            _ = watermarked_model(
                input_ids=trigger_inputs["input_ids"][:1],
                attention_mask=trigger_inputs["attention_mask"][:1]
            )
        hidden_size = dummy_hook.features.shape[-1]

        dummy_hook.close()
        
        print(f"Hidden size: {hidden_size}")
        
        proj_net = ProjectionNet(hidden_size, self.config["watermark_len"]).to(self.device)
        proj_net.train()
        
        # 4. Optimiseurs
        optimizer_model = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        optimizer_proj = torch.optim.AdamW(proj_net.parameters(), lr=self.config["lr_proj"])
        
        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()
        
        # 5. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            correct = 0
            total = 0
            
            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                # --- A. Entrainement ProjectionNet ---
                optimizer_proj.zero_grad()
                
                # Features originales -> random watermark
                orig_layer = self._get_target_layer(original_model, self.config["layer_name"])
                hook_orig = FeatureHook(orig_layer)
                with torch.no_grad():
                    _ = original_model(
                        input_ids=trigger_inputs["input_ids"],
                        attention_mask=trigger_inputs["attention_mask"]
                    )
                feat_orig = hook_orig.features
                hook_orig.close()
                
                # Features tatouees -> target watermark
                wat_layer = self._get_target_layer(watermarked_model, self.config["layer_name"])
                hook_wat = FeatureHook(wat_layer)
                _ = watermarked_model(
                    input_ids=trigger_inputs["input_ids"],
                    attention_mask=trigger_inputs["attention_mask"]
                )
                feat_wat = hook_wat.features
                hook_wat.close()
                
                # Projection losses
                pred_orig = proj_net(feat_orig.detach())
                pred_wat = proj_net(feat_wat.detach())
                
                l_proj_orig = bce_loss(pred_orig.mean(dim=0), random_wm)
                l_proj_wat = bce_loss(pred_wat.mean(dim=0), target_wm)
                l_proj = l_proj_orig + l_proj_wat
                
                l_proj.backward()
                optimizer_proj.step()
                
                # --- B. Entrainement Modele Tatoue ---
                optimizer_model.zero_grad()
                
                # Tache principale (classification)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                l_main = ce_loss(outputs.logits, labels)
                
                # Tache watermark (sur trigger set)
                hook_wat2 = FeatureHook(wat_layer)
                _ = watermarked_model(
                    input_ids=trigger_inputs["input_ids"],
                    attention_mask=trigger_inputs["attention_mask"]
                )
                feat_wat2 = hook_wat2.features
                hook_wat2.close()
                
                pred_wat2 = proj_net(feat_wat2)
                l_wat = bce_loss(pred_wat2.mean(dim=0), target_wm)
                
                # Loss totale
                l_total = l_main + self.config["lambda_wat"] * l_wat
                
                l_total.backward()
                optimizer_model.step()
                
                # Metrics
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                ber = self._compute_ber(pred_wat2.mean(dim=0), target_wm)
                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}", L_Wat=f"{l_wat.item():.3f}",
                                BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
            "trigger_inputs": trigger_inputs,
            "target_wm": target_wm,
            "random_wm": random_wm,
            "proj_net": proj_net,
            "watermarked_model": watermarked_model,
            "original_model": original_model
        }
        
        final_ber, _ = self.extract(watermarked_model)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]
        
        suspect_model.eval()
        proj_net = self.saved_keys["proj_net"]
        proj_net.eval()
        
        trigger_inputs = self.saved_keys["trigger_inputs"]
        target_wm = self.saved_keys["target_wm"]
        
        # Hook sur le modele suspect
        target_layer = self._get_target_layer(suspect_model, self.config["layer_name"])
        hook = FeatureHook(target_layer)
        
        # Passage du trigger set
        with torch.no_grad():
            _ = suspect_model(
                input_ids=trigger_inputs["input_ids"],
                attention_mask=trigger_inputs["attention_mask"]
            )
        
        features = hook.features
        hook.close()
        
        # Projection & BER
        wm_pred = proj_net(features).mean(dim=0)
        ber = self._compute_ber(wm_pred, target_wm)
        
        suspect_model.train()
        proj_net.train()
        
        print(f"BER Extrait : {ber:.2f}")
        return ber, wm_pred
    
    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()

print("Classes chargees (DictionSentiment, FeatureHook, ProjectionNet)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classes chargees (DictionSentiment, FeatureHook, ProjectionNet)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker
diction = DictionSentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
diction.list_layers()

# Tokenization
def tokenize_function(examples):
    return diction.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2545.56it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias         

Couches disponibles (modules):
  bert
  bert.embeddings
  bert.embeddings.word_embeddings
  bert.embeddings.position_embeddings
  bert.embeddings.token_type_embeddings
  bert.embeddings.LayerNorm
  bert.embeddings.dropout
  bert.encoder
  bert.encoder.layer
  bert.encoder.layer.0
  bert.encoder.layer.0.attention
  bert.encoder.layer.0.attention.self
  bert.encoder.layer.0.attention.self.query
  bert.encoder.layer.0.attention.self.key
  bert.encoder.layer.0.attention.self.value
  bert.encoder.layer.0.attention.self.dropout
  bert.encoder.layer.0.attention.output
  bert.encoder.layer.0.attention.output.dense
  bert.encoder.layer.0.attention.output.LayerNorm
  bert.encoder.layer.0.attention.output.dropout
  bert.encoder.layer.0.intermediate
  bert.encoder.layer.0.intermediate.dense
  bert.encoder.layer.0.intermediate.intermediate_act_fn
  bert.encoder.layer.0.output
  bert.encoder.layer.0.output.dense
  bert.encoder.layer.0.output.LayerNorm
  bert.encoder.layer.0.output.dropout
  bert.enc

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 34885.96 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:
# --- Embedding du Watermark ---
#
# # Configuration
# diction.config["epochs"] = 5
# diction.config["lambda_wat"] = 10.0
# diction.config["watermark_len"] = 64
# diction.config["trigger_size"] = 32

# Lancer l'embedding
watermarked_model = diction.embed(
    train_dataloader, 
    eval_dataloader,
    trigger_dataset=tokenized_dataset["train"]
)

--- Demarrage Embedding DICTION (bert.encoder.layer.1.output) ---
Trigger set size: 32
Watermark length: 64 bits
Hidden size: 128


Epoch 1/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:08<00:00, 122.70it/s, Acc=78.2%, BER=0.00, L_Main=0.291, L_Wat=0.000]


Epoch 1 | Eval Accuracy: 80.85%


Epoch 2/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 116.11it/s, Acc=88.4%, BER=0.00, L_Main=0.274, L_Wat=0.000]


Epoch 2 | Eval Accuracy: 82.00%


...

Epoch 8 | Eval Accuracy: 80.85%


Epoch 9/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:09<00:00, 116.99it/s, Acc=95.4%, BER=0.00, L_Main=0.093, L_Wat=0.000]


Epoch 9 | Eval Accuracy: 81.19%


Epoch 10/10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:08<00:00, 120.71it/s, Acc=95.8%, BER=0.00, L_Main=0.392, L_Wat=0.000]


Epoch 10 | Eval Accuracy: 79.13%
BER Extrait : 0.00

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = diction.extract(watermarked_model)

# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")
model_id="prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device=diction.device)

ber_fresh, _ = diction.extract(fresh_model)

print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:
BER Extrait : 0.00

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2059.63it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias         

BER Extrait : 0.52

Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.52 (attendu: ~0.5)


In [5]:
# --- Fonction de Distillation (Attaque) ---
def compute_stats(features):
    mean = features.mean(dim=0)
    var = features.var(dim=0)
    return mean, var



def run_distillation_attack(diction_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les activations) survit.
    """
    device = diction_obj.device
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    teacher_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    # 1. Teacher (Gele)
    # teacher_model = diction_obj.saved_keys["watermarked_model"]

    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False


    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()


    
    # Verification initiale
    print("\n[Check] BER Teacher:")
    teacher_ber, _ = diction_obj.extract(teacher_model)
    print("[Check] BER Student (Avant):")
    student_ber, _ = diction_obj.extract(student_model)
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        student_model.train()
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()
            teacher_layer = diction._get_target_layer(teacher_model, diction.config["layer_name"])
            hook = FeatureHook(teacher_layer)
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)
                t_features = hook.features
            hook.close()

            student_layer = diction._get_target_layer(student_model, diction.config["layer_name"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()
            # print(f"Teacher features shape: {t_features.shape} | Student features shape: {s_features.shape}")
            # t_mean, t_var = compute_stats(t_features)
            # s_mean, s_var = compute_stats(s_features)
            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl + 0.1*loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob= diction_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = diction_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% err_wat: {nn.BCELoss()(pred_wm_prob, diction_obj.saved_keys["target_wm"]).item()}")
    
    return student_model, history

In [ ]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    diction, 
    train_dataloader, 
    eval_dataloader,
    epochs=1000
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 3165.21it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias         


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2372.72it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias         


[Check] BER Teacher:
BER Extrait : 0.52
[Check] BER Student (Avant):
BER Extrait : 0.52

--- Distillation (1000 epochs) ---


...

Epoch 672: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 188.61it/s, Loss=0.0025]


BER Extrait : 0.52
Epoch 672 | Loss: 0.0018 | BER: 0.52 | Acc: 49.1% err_wat: 8.348127365112305


Epoch 673: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 191.12it/s, Loss=0.0028]


BER Extrait : 0.52
Epoch 673 | Loss: 0.0018 | BER: 0.52 | Acc: 49.1% err_wat: 8.33056640625


Epoch 674:  77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                      | 816/1053 [00:04<00:01, 181.60it/s, Loss=0.0018]

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
teacher_ber, _ = diction.extract(diction.saved_keys['watermarked_model'])
print(f"BER Teacher (watermarke): {teacher_ber:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque Diction n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque Diction a survecu a la distillation (BER faible)")